In [0]:
%run "../commons/common_functions"

In [0]:
%run "../commons/configuration"

In [0]:
dbutils.widgets.text("env_name","DEV")
p_env_name = dbutils.widgets.get("env_name")

In [0]:
from pyspark.sql.functions import current_timestamp
from pyspark.sql.functions import to_timestamp, concat, lit, col
from pyspark.sql.types import *

name_schema = StructType([StructField("forename", StringType(), True), StructField("surname", StringType(), True)])

driver_schema = StructType([StructField("driverId", IntegerType(), False), 
                            StructField("driverRef", StringType(), True), 
                            StructField("number", IntegerType(), True), 
                            StructField("code", StringType(), True),
                            StructField("name", name_schema),
                            StructField("dob", DateType(), True),
                            StructField("nationality", StringType(), True),
                            StructField("url", StringType(), True),])




In [0]:
drivers_df = spark.read.json(f"{raw_mount_path}/drivers.json", schema=driver_schema)\
    .withColumnRenamed("driverId", "driver_id")\
    .withColumnRenamed("driverRef","driver_ref")\
    .drop("url")\
    .withColumn("name", concat(col("name.forename"), lit(","), col("name.surname")))\
    .withColumn("data_source",lit(p_env_name))

In [0]:
drivers_df_v2 = add_ingestion_date(drivers_df)

In [0]:
#drivers_df_v2.write.parquet(f"{processed_mount_path}/drivers", mode="overwrite")
drivers_df_v2.write.mode("overwrite").format("parquet").saveAsTable("processed.drivers")

In [0]:
dbutils.notebook.exit("Success")